In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Input, Conv3D, MaxPool3D, BatchNormalization, Dropout, Flatten, Dense, Activation
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [ ]:
X = []
Y = []

datadir = 'Dynamic_Hand_Gesture_Recognition/dataset'
for classes in os.listdir(datadir):
  path = os.path.join(datadir, classes)
  for folder in os.listdir(path):
    m = np.zeros(shape=(30, 128, 128, 1))
    imgs = os.path.join(path, folder)
    for i, img in enumerate(os.listdir(imgs)):
      image = cv2.imread(os.path.join(imgs, img))
      image = image/255.
      m[i, :, :, :] = image
    X.append(m)
    Y.append(classes)

print(len(X))
print(len(Y))

In [ ]:
encoder = LabelEncoder()
Y = encoder.fit_transform(encoder)

Y = np.array(Y)
X = np.array(X)
X = np.expand_dims(X, axis=-1)

Y = to_categorical(Y)

print(X.shape)
print(Y.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)

In [ ]:
unique_train, count_train = np.unique(Y_train, return_counts=True)
figure = plt.figure(figsize=(7, 7))
sns.barplot(unique_train, count_train).set_title('Number of Images per category in Training Set')
plt.show()

In [ ]:
unique_test, count_test = np.unique(Y_test, return_counts=True)
figure = plt.figure(figsize=(7, 7))
sns.barplot(unique_test, count_test).set_title('Number of Images per category in Test Set')
plt.show()

In [ ]:
model = Sequential()
model.add(Input(shape=(30, 128, 128, 1)))
model.add(Conv3D(16, (5, 5, 5), activation='relu', kernel_initializer=glorot_uniform(seed=0), name='conv1'))
model.add(MaxPool3D((2, 2, 2), padding='same'))
model.add(BatchNormalization())
model.add(Conv3D(32, (3, 3, 3), activation='relu', kernel_initializer=glorot_uniform(seed=0), name='conv2'))
model.add(MaxPool3D((2, 2, 2), padding='same'))
model.add(BatchNormalization())
model.add(Conv3D(64, (3, 3, 3), activation='relu', kernel_initializer=glorot_uniform(seed=0), name='conv3'))
model.add(MaxPool3D((2, 2, 2), padding='same'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer=glorot_uniform(seed=0), name='fc1'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu', kernel_initializer=glorot_uniform(seed=0), name='fc2'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax', kernel_initializer=glorot_uniform(seed=0), name='fc3'))

opt = Adam()

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
checkpoint = ModelCheckpoint('/content/drive/MyDrive/hand_gesture_recognition_v1.h5', monitor='val_loss', save_best_only=True, verbose=1, mode='min')
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)

def step_decay(epoch):
    initial_learning_rate = 0.001
    dropEvery = 10
    factor = 0.5
    lr = initial_learning_rate*(factor**np.floor((1 + epoch)/dropEvery))
    return float(lr)

callbacks = [checkpoint, LR]
# callbacks = [checkpoint, LearningRateScheduler(step_decay)]

hist = model.fit(X_train, Y_train, batch_size=32, epoch=20, validation_data=(X_test, Y_test), callbacks=callbacks)

In [ ]:
figure1 = plt.figure(figsize=(10, 10))
plt.plot(hist.history['accuracy'], label='Train_accuracy')
plt.plot(hist.history['val_accuracy'], label='Test_accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc="upper left")
plt.show()

figure2 = plt.figure(figsize=(10, 10))
plt.plot(hist.history['loss'], label='Train_loss')
plt.plot(hist.history['val_loss'], label='Test_loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="upper left")
plt.show()

In [ ]:
y_pred_test = model.predict(X_test)
y_pred_test = np.argmax(y_pred_test, axis=1)
Y_test_copy = np.argmax(Y_test, axis=1)
print(classification_report(Y_test_copy, y_pred_test))

In [ ]:
matrix = confusion_matrix(Y_test_copy, y_pred_test)
figure = plt.figure(figsize=(7, 7))
df = pd.DataFrame(matrix, index=[0, 1, 2, 3, 4, 5, 6, 7], columns=[0, 1, 2, 3, 4, 5, 6, 7])
sns.heatmap(df, annot=True, fmg='d')
plt.show()

In [ ]:
def batch_generator(t, batch_number, batch_size):
  d = {''}
  batch_data = np.zeros(shape=(batch_size, 30, 128, 128, 1))
  batch_label = np.zeros(shape=(batch_size, 8))
  for folder in range(batch_size):
    imgs = os.listdir('dataset/' + t[folder + (batch_number*batch_size)])
    for i, img in enumerate(imgs):
      image = cv2.imread(os.path.join('dataset/' + t[folder + (batch_number*batch_size)], img))
      image = image/255.
      batch_data[folder, i, :, :, :] = image
    batch_label[folder, d[t[folder + (batch_number*batch_size)].split('_')[0]]] = 1

In [ ]:
def generator(folder_list, batch_size):
  while True:
    t = folder_list
    num_batches = int(len(folder_list)/batch_size)
    for batch in range(num_batches):
      yield batch_generator(t, batch, batch_size)

    if len(folder_list)%batch_size == 0:
      batch_size = len(folder_list)%batch_size
      yield batch_generator(t, batch, batch_size) 

In [ ]:
folder_names = os.listdir('dataset')
folder_names = np.random.permutation(folder_names)

train_folders = folder_names[:0.8*len(folder_names)]
test_folders = folder_names[0.8*len(folder_names):]

train_generator = generator(train_folders, 32)
test_generator = generator(test_folders, 32)

number_of_train_sequences = 0.8*len(folder_names)
number_of_test_sequences = 0.2*len(folder_names)

if (number_of_train_sequences%32) == 0:
    steps_per_epoch = int(number_of_train_sequences/32)
else:
    steps_per_epoch = (number_of_train_sequences//32) + 1

if (number_of_test_sequences%32) == 0:
    validation_steps = int(number_of_test_sequences/32)
else:
    validation_steps = (number_of_test_sequences//32) + 1

In [ ]:
model = Sequential()
model.add(Input(shape=(30, 128, 128, 1)))
model.add(Conv3D(16, (5, 5, 5), activation='relu', kernel_initializer=glorot_uniform(seed=0), name='conv1'))
model.add(MaxPool3D((2, 2, 2), padding='same'))
model.add(BatchNormalization())
model.add(Conv3D(32, (3, 3, 3), activation='relu', kernel_initializer=glorot_uniform(seed=0), name='conv2'))
model.add(MaxPool3D((2, 2, 2), padding='same'))
model.add(BatchNormalization())
model.add(Conv3D(64, (3, 3, 3), activation='relu', kernel_initializer=glorot_uniform(seed=0), name='conv3'))
model.add(MaxPool3D((2, 2, 2), padding='same'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer=glorot_uniform(seed=0), name='fc1'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu', kernel_initializer=glorot_uniform(seed=0), name='fc2'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax', kernel_initializer=glorot_uniform(seed=0), name='fc3'))

opt = Adam()

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
checkpoint = ModelCheckpoint('/content/drive/MyDrive/hand_gesture_recognition_v1.h5', monitor='val_loss', save_best_only=True, verbose=1, mode='min')
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)

def step_decay(epoch):
    initial_learning_rate = 0.001
    dropEvery = 10
    factor = 0.5
    lr = initial_learning_rate*(factor**np.floor((1 + epoch)/dropEvery))
    return float(lr)

callbacks = [checkpoint, LR]
# callbacks = [checkpoint, LearningRateScheduler(step_decay)]

hist = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, 
                           epochs=num_epochs, verbose=1, callbacks=callbacks,
                           validation_data=val_generator, 
                           validation_steps=validation_steps,
                           class_weight=None, workers=1, initial_epoch=0)

In [ ]:
figure1 = plt.figure(figsize=(10, 10))
plt.plot(hist.history['accuracy'], label='Train_accuracy')
plt.plot(hist.history['val_accuracy'], label='Test_accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc="upper left")
plt.show()

figure2 = plt.figure(figsize=(10, 10))
plt.plot(hist.history['loss'], label='Train_loss')
plt.plot(hist.history['val_loss'], label='Test_loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="upper left")
plt.show()